In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['RB', 'Rb']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('retinoblastoma', 1927),
 ('retinoblastoma protein', 1161),
 ('retinoblastoma tumor suppressor protein', 132),
 ('rose bengal', 112),
 ('retinoblastoma gene product', 74),
 ('retinoblastoma tumor suppressor', 60),
 ('retinoblastoma gene', 50),
 ('rohon beard', 41),
 ('respiratory burst', 29),
 ('reticulate body', 27),
 ('rhodamine b', 25),
 ('rice bran', 20),
 ('retinoblastoma tumor suppressor gene', 18),
 ('retinoblastoma susceptibility gene product', 17),
 ('retinoblastoma susceptibility gene', 14),
 ('rubidium', 14),
 ('retinoblastoma susceptibility protein', 14),
 ('retinoblastoma tumour suppressor protein', 14),
 ('retinoblastoma 1', 12),
 ('retinoblastoma susceptibility', 12),
 ('retinoblastoma associated protein', 9),
 ('repeat breeder', 8),
 ('rod bipolar', 8),
 ('rule based', 8),
 ('red berry', 6),
 ('roaming beat', 6),
 ('resistance breaking', 6),
 ('raspberry', 6),
 ('red bull', 5),
 ('radix bupleuri', 5),
 ('reactive blue', 5),
 ('retigeric acid b', 5),
 ('rb1', 5),
 ('re

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'MESH:D012175': 'Retinoblastoma',
 'HGNC:9884': 'RB1',
 'CHEBI:CHEBI:52261': 'ROSE-BENGAL',
 'GO:GO:0051726': 'regulation of cell cycle',
 'HGNC:28135': 'TCHP',
 'GO:GO:0045730': 'RESPIRATORY-BURST',
 'CHEBI:CHEBI:52334': 'rhodamine B',
 'MESH:D016161': 'Genes, Retinoblastoma',
 'MESH:D016147': 'Genes, Tumor Suppressor',
 'MESH:D005796': 'Genes',
 'MESH:D012119': 'Respiration',
 'CHEBI:CHEBI:17015': 'riboflavin',
 'CHEBI:CHEBI:31319': 'bufogenin',
 'MESH:D006471': 'Gastrointestinal Hemorrhage',
 'MESH:D001766': 'Blindness',
 'GO:GO:0005488': 'binding',
 'CHEBI:CHEBI:27560': 'boron atom',
 'GO:GO:0005623': 'cell',
 'CHEBI:CHEBI:33322': 'RUBIDIUM'}

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-04 01:40:53] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'radix bupleuri': 'ungrounded',
  'ras binding': 'ungrounded',
  'raspberry': 'ungrounded',
  'rb1': 'HGNC:9884',
  'reactive blue': 'ungrounded',
  'rectal bleeding': 'MESH:D006471',
  'red berry': 'ungrounded',
  'red blue': 'ungrounded',
  'red bull': 'ungrounded',
  'reduced body': 'ungrounded',
  'repeat bout': 'ungrounded',
  'repeat breeder': 'ungrounded',
  'repetition blindness': 'ungrounded',
  'resibufogenin': 'CHEBI:CHEBI:31319',
  'residual body': 'ungrounded',
  'resistance breaking': 'ungrounded',
  'resistance breathing': 'ungrounded',
  'respiratory burst': 'GO:GO:0045730',
  'resting b': 'ungrounded',
  'reticulate body': 'ungrounded',
  'retigeric acid b': 'ungrounded',
  'retinoblas toma': 'HGNC:9884',
  'retinoblastoma': 'HGNC:9884',
  'retinoblastoma 1': 'HGNC:9884',
  'retinoblastoma associated protein': 'HGNC:9884',
  'retinoblastoma gene': 'HGNC:9884',
  'retinoblastoma gene product': 'HGNC:9884',
  'retinoblastoma protein': 'HGNC:9884',
  'retinoblastoma sus

In [11]:
grounding_map, names, pos_labels = [{'radix bupleuri': 'ungrounded',
  'ras binding': 'ungrounded',
  'raspberry': 'ungrounded',
  'rb1': 'HGNC:9884',
  'reactive blue': 'ungrounded',
  'rectal bleeding': 'MESH:D006471',
  'red berry': 'ungrounded',
  'red blue': 'ungrounded',
  'red bull': 'ungrounded',
  'reduced body': 'ungrounded',
  'repeat bout': 'ungrounded',
  'repeat breeder': 'ungrounded',
  'repetition blindness': 'ungrounded',
  'resibufogenin': 'CHEBI:CHEBI:31319',
  'residual body': 'ungrounded',
  'resistance breaking': 'ungrounded',
  'resistance breathing': 'ungrounded',
  'respiratory burst': 'GO:GO:0045730',
  'resting b': 'ungrounded',
  'reticulate body': 'ungrounded',
  'retigeric acid b': 'ungrounded',
  'retinoblas toma': 'HGNC:9884',
  'retinoblastoma': 'HGNC:9884',
  'retinoblastoma 1': 'HGNC:9884',
  'retinoblastoma associated protein': 'HGNC:9884',
  'retinoblastoma gene': 'HGNC:9884',
  'retinoblastoma gene product': 'HGNC:9884',
  'retinoblastoma protein': 'HGNC:9884',
  'retinoblastoma susceptibility': 'ungrounded',
  'retinoblastoma susceptibility gene': 'HGNC:9884',
  'retinoblastoma susceptibility gene product': 'HGNC:9884',
  'retinoblastoma susceptibility protein': 'HGNC:9884',
  'retinoblastoma tumor suppressor': 'HGNC:9884',
  'retinoblastoma tumor suppressor gene': 'HGNC:9884',
  'retinoblastoma tumor suppressor gene product': 'HGNC:9884',
  'retinoblastoma tumor suppressor protein': 'HGNC:9884',
  'retinoblastoma tumour suppressor': 'HGNC:9884',
  'retinoblastoma tumour suppressor protein': 'ungrounded',
  'retired breeder': 'ungrounded',
  'rho binding domain': 'ungrounded',
  'rhodamine b': 'CHEBI:CHEBI:52334',
  'riboflavin': 'CHEBI:CHEBI:17015',
  'rice bran': 'ungrounded',
  'roaming beat': 'ungrounded',
  'rod bipolar': 'ungrounded',
  'rohon beard': 'CL:CL:0000247',
  'rohon beard cell': 'CL:CL:0000247',
  'rose bengal': 'CHEBI:CHEBI:52261',
  'rubidium': 'CHEBI:CHEBI:33322',
  'ruffled border': 'ungrounded',
  'rule based': 'ungrounded',
  'rule breaking': 'ungrounded'},
 {'HGNC:9884': 'RB1',
  'MESH:D006471': 'Gastrointestinal Hemorrhage',
  'CHEBI:CHEBI:31319': 'bufogenin',
  'GO:GO:0045730': 'respiratory burst',
  'CHEBI:CHEBI:52334': 'rhodamine B',
  'CHEBI:CHEBI:17015': 'riboflavin',
  'CL:CL:0000247': 'Rohon-Beard neuron',
  'CHEBI:CHEBI:52261': 'rose bengal',
  'CHEBI:CHEBI:33322': 'rubidium atom'},
 ['CHEBI:CHEBI:52261',
  'CHEBI:CHEBI:52334',
  'CL:CL:0000247',
  'GO:GO:0045730',
  'HGNC:9884']]

In [12]:
excluded_longforms = []

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {}

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [17]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-04 01:48:40] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-04 01:49:39] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9726783647375781 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [18]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:52261': 81,
  'HGNC:9884': 2967,
  'GO:GO:0045730': 27,
  'ungrounded': 119,
  'CL:CL:0000247': 26,
  'CHEBI:CHEBI:52334': 19,
  'CHEBI:CHEBI:17015': 3,
  'MESH:D006471': 2,
  'CHEBI:CHEBI:31319': 4,
  'CHEBI:CHEBI:33322': 14},
 'f1': {'mean': 0.972678, 'std': 0.005484},
 'precision': {'mean': 0.962761, 'std': 0.008155},
 'recall': {'mean': 0.984936, 'std': 0.003598},
 'CL:CL:0000247': {'f1': {'mean': 0.937374, 'std': 0.051663},
  'pr': {'mean': 0.92, 'std': 0.09798},
  'rc': {'mean': 0.966667, 'std': 0.066667}},
 'GO:GO:0045730': {'f1': {'mean': 0.771111, 'std': 0.07281},
  'pr': {'mean': 0.633333, 'std': 0.098883},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.634218, 'std': 0.03751},
  'pr': {'mean': 0.513406, 'std': 0.056788},
  'rc': {'mean': 0.83977, 'std': 0.041335}},
 'CHEBI:CHEBI:17015': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'CHEBI:CHEBI:33322': {'f1'

In [19]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [20]:
disamb.dump(model_name, results_path)

In [21]:
print(disamb.info())

Disambiguation model for RB, and Rb

Produces the disambiguations:
	Gastrointestinal Hemorrhage	MESH:D006471
	RB1*	HGNC:9884
	Rohon-Beard neuron*	CL:CL:0000247
	bufogenin	CHEBI:CHEBI:31319
	respiratory burst*	GO:GO:0045730
	rhodamine B*	CHEBI:CHEBI:52334
	riboflavin	CHEBI:CHEBI:17015
	rose bengal*	CHEBI:CHEBI:52261
	rubidium atom	CHEBI:CHEBI:33322

Class level metrics:
--------------------
Grounding                  	Count	F1     
                        RB1*	2967	0.98209
                 Ungrounded	 119	0.63422
                rose bengal*	  81	0.83326
          respiratory burst*	  27	0.77111
         Rohon-Beard neuron*	  26	0.93737
                rhodamine B*	  19	0.43333
              rubidium atom	  14	0.23333
                  bufogenin	   4	    0.0
                 riboflavin	   3	    0.0
Gastrointestinal Hemorrhage	   2	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.97268
	Precision:	0.96276
	Recall:		0.98494

* Positive labels
See Docstring for explanation



In [22]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1